In [1]:
import os

In [2]:
import pandas as pd
import numpy as np

np.set_printoptions(precision=3, suppress=True)

pd.options.display.float_format = lambda num: f'{num:.4f}'

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={"figure.figsize": (12, 8)})

In [4]:
from utils.ingest import read_data
from fixed_effects import within_group
from gmm import lagged_gmm

#### Constants

In [5]:
data_path = "data/hw3.xls"

### Data inject

In [6]:
data = read_data(data_path)

### 1. Stationarity issues

#### (a)



### 2. Homogenous panel

#### (a)

The panel is:
- homogenous
- no heterogeneity in the slope parameters
- no endogeneity
- no cross-sectional dependence

Given the assumptions, the suitable estimator is the Withing Group (or Fixed Effects) estimators.

In [7]:
regs = ["d(lnY)", "INF"]
beta, fixed_effects, resid, var, dw = within_group(data, "S/Y", regs, lags = 0)


Withing regression:  

--- β:

d(lnY)   0.293056
INF      0.179817, 


--- Standard errors:

         d(lnY)      INF
d(lnY) 0.017580 0.000416
INF    0.000416 0.003499
        
--- Durbin-Watson
0.0426  



#### 3. Dynamic homogenous panels

Within regressor with lagged value

In [8]:
regs = ["d(lnY)", "INF"]
beta, fixed_effects, resid, var, dw = within_group(data, "S/Y", regs, lags = 1, title="Lagged Within regression")


Lagged Within regression:  

--- β:

d(lnY)      0.023697
INF         0.021070
lag_1_S/Y   0.848010, 


--- Standard errors:

             d(lnY)       INF  lag_1_S/Y
d(lnY)     0.001158  0.000059  -0.000196
INF        0.000059  0.000318  -0.000157
lag_1_S/Y -0.000196 -0.000157   0.001078
        
--- Durbin-Watson
1.1803  



#### 4. Enogeneity in homogenous dynamic panel

In [9]:
lags = 2

In [10]:
_ = lagged_gmm(
        data, "S/Y",
        regressors=["d(lnY)", "INF"], lag_inst=lags
    )



IV Estimation:  

--- β:

d(lnY)      -1.331277
INF         -0.459295
lag_1_S/Y    3.493394, 


--- Standard errors:

              d(lnY)       INF  lag_1_S/Y
d(lnY)      5.459203  2.315545 -13.229239
INF         2.315545  1.009244  -5.721096
lag_1_S/Y -13.229239 -5.721096  32.702864
        
--- Durbin-Watson
0.7373  



In [11]:
_ = lagged_gmm(
        data, "S/Y",
        regressors=["d(lnY)", "INF"], lag_inst=lags,
        gmm=True
    )


GMM Estimation:  

--- β:

d(lnY)      -1.907793
INF         -0.710240
lag_1_S/Y    4.909449, 


--- Standard errors:

            d(lnY)      INF  lag_1_S/Y
d(lnY)    0.000000 0.000000   0.000000
INF       0.000000 0.000000   0.000000
lag_1_S/Y 0.000000 0.000000   0.000000
        
--- Overidentifying Restrictions
0.3887  p=0.4670

--- Durbin-Watson
0.6272  



#### 5. Cross-sectional dependence in a homogeneous dynamic panel.

In [16]:
from itertools import permutations

In [22]:
np.isclose(np.zeros((2,2))[1, 1], 0.)

True